# Triage v1 — Selección no destructiva para etiquetado

Este notebook toma `etiquetas_v1.csv` y genera **dos archivos**:

- `etiquetas_v1_TRIAGE.csv` → lote reducido (~objetivo 2.000) priorizado para **etiquetar**.
- `etiquetas_v1_TRIAGE_pool.csv` → el **resto** (nada se borra).

Decisiones clave:
- **No** tocamos el CSV original.
- Priorizamos combinaciones de `origen_flags` con mayor precisión.
- **Colapsamos** por norma (`numero_norma`) para evitar repeticiones.
- **Top-K por Boletín** (`origen_pdf`) para no sobre-representar PDFs “ricos”.
- Aplicamos **cuotas por combinación** para llegar al tamaño objetivo.
- Añadimos una **muestra de seguridad** de buckets excluidos (opcional).


In [1]:
# === 1) Entorno ===
from google.colab import drive
drive.mount('/content/drive')

import os, re, math, random
from datetime import datetime
import numpy as np
import pandas as pd

print('Entorno listo ✅')

Mounted at /content/drive
Entorno listo ✅


In [11]:
# === 2) Configuración (EDITAR si cambia tu ruta) ===
BASE = "/content/drive/MyDrive/IA/Proyectos/Análisis Boletín Oficial/boletin-ml"
CSV_INPUT  = f"{BASE}/data/labels/etiquetas_v1.csv"
CSV_TRIAGE = f"{BASE}/data/labels/etiquetas_v1_TRIAGE.csv"
CSV_POOL   = f"{BASE}/data/labels/etiquetas_v1_TRIAGE_pool.csv"

# Combinaciones prioritarias (las que vos filtraste):
PRIORITY_BUCKETS = [
    "VERBO;KEYWORD;NORMAR",
    "VERBO;KEYWORD",
    "VERBO;NORMAR",
]

# Tamaño objetivo del lote etiquetable
TARGET_SIZE = 2000  # cambiá si querés otro tamaño

# Cuotas por bucket (suman ~1.0). Ajustalas si hace falta.
BUCKET_QUOTAS = {
    "VERBO;KEYWORD;NORMAR": 0.40,
    "VERBO;NORMAR":         0.30,
    "VERBO;KEYWORD":        0.30,
}

# Top K por Boletín (para no inundar el lote con un solo PDF)
TOPK_PER_PDF = 20

# Safety sample (muestra de seguridad de buckets excluidos)
INCLUDE_SAFETY_SAMPLE = True
SAFETY_SIZE = 250  # p.ej., 250 ejemplos aleatorios de buckets no prioritarios

print('Configuración cargada ✅')

Configuración cargada ✅


In [12]:
# === 3) Carga ===
assert os.path.exists(CSV_INPUT), f"No se encontró el archivo: {CSV_INPUT}"
df = pd.read_csv(CSV_INPUT, dtype=str, keep_default_na=False)
print(df.shape)
df.head(2)

(27250, 19)


,id,split,fecha,anio,boletin_nro,origen_pdf,fuente_url,organismo_emisor,tipo_figura_normativa,titulo,fragmento,label,anotador,notas,is_ambiguo,deroga_modifica_ref,numero_norma,temas,origen_flags
0,20180102-p001-s004-e008-h40bf789b,train,2018-01-02,2018,,20180102.pdf,,,,,Olga García Carricaburu de D\'Agostino...........,,,,0,0,Ley N° 5919,sistema de autoprotección,VERBO;KEYWORD;NORMAR
1,20180102-p022-s006-e012-heb3b4d6f,train,2018-01-02,2018,,20180102.pdf,,,,,José Luis Estrada................................,,,,0,0,N° 30231156-,,KEYWORD


## 4) Columnas auxiliares (no destructivas)
- `has_VERBO`, `has_KEYWORD`, `has_NORMAR`: detectan flags en `origen_flags`.
- `bucket`: combinación canónica de flags presentes.
- `score`: prioriza por señales fuertes.
- `group_norma`: clave para colapsar repeticiones (usa `numero_norma` si existe; si no, `origen_pdf`).
- `rand`: aleatorio para desempates/muestras.


In [13]:
def norm_text(x: str) -> str:
    x = (x or "").strip().lower()
    x = re.sub(r"\s+", " ", x)
    return x

flags = df.get('origen_flags', '').fillna('')
f_verbo   = flags.str.contains(r"\bVERBO\b", regex=True)
f_keyword = flags.str.contains(r"\bKEYWORD\b", regex=True)
f_normar  = flags.str.contains(r"\bNORMAR\b", regex=True)

df['has_VERBO']   = f_verbo.astype(int)
df['has_KEYWORD'] = f_keyword.astype(int)
df['has_NORMAR']  = f_normar.astype(int)

# bucket canónico (orden VERBO;KEYWORD;NORMAR si están presentes)
def make_bucket(row):
    parts = []
    if row['has_VERBO']   == 1: parts.append('VERBO')
    if row['has_KEYWORD'] == 1: parts.append('KEYWORD')
    if row['has_NORMAR']  == 1: parts.append('NORMAR')
    return ";".join(parts)
df['bucket'] = df.apply(make_bucket, axis=1)

# score: +3 NORMAR, +2 VERBO, +1 KEYWORD, +2 si contiene ARTÍCULO 1 / RESUELVE / DISPONE
frag = df.get('fragmento', '').fillna('')
has_art1   = frag.str.contains(r"ART[ÍI]CULO\s*1", case=False, regex=True)
has_resolv = frag.str.contains(r"RESUELVE", case=False, regex=False)
has_disp   = frag.str.contains(r"DISPONE", case=False, regex=False)
has_decreta = frag.str.contains(r"\bDECRETA\b", case=False, regex=True)
has_sanciona_ley = frag.str.contains(r"sanciona\s+con\s+fuerza\s+de\s+ley", case=False, regex=True)

df['score'] = (
    3*df['has_NORMAR'] +          # cita normativa formal
    2*df['has_VERBO'] +           # verbo normativo
    1*df['has_KEYWORD'] +         # keyword de dominio
    2*(has_art1 | has_resolv | has_disp).astype(int) +  # señales de secciones decisorias
    2*has_decreta.astype(int) +   # DECRETA
    3*has_sanciona_ley.astype(int)  # Legislatura sanciona con fuerza de Ley
)

# group_norma: numero_norma si existe, si no, origen_pdf
num_norma = df.get('numero_norma', '').fillna('').map(norm_text)
orig_pdf  = df.get('origen_pdf', '').fillna('').map(norm_text)

import hashlib

def sha1_norm(s: str) -> str:
    return hashlib.sha1(norm_text(s).encode('utf-8')).hexdigest()

# ✅ Opción B: si hay numero_norma agrupamos por eso; si NO hay,
# colapsamos por el *propio fragmento normalizado* (no por PDF entero).
df["group_norma"] = np.where(
    num_norma.str.len() > 0,
    num_norma,                # agrupa por número de norma
    frag.map(sha1_norm)       # si falta número, agrupa por contenido
)

np.random.seed(42)
df['rand'] = np.random.rand(len(df))

df[['origen_flags','bucket','score','group_norma']].head(10)

,origen_flags,bucket,score,group_norma
0,VERBO;KEYWORD;NORMAR,VERBO;KEYWORD;NORMAR,6,ley n° 5919
1,KEYWORD,KEYWORD,1,n° 30231156-
2,KEYWORD,KEYWORD,1,n° 25294772-
3,KEYWORD,KEYWORD,1,n° 26581270-
4,KEYWORD,KEYWORD,1,n° 27707530-
5,KEYWORD,KEYWORD,1,n° 29350712-
6,KEYWORD,KEYWORD,6,nº 5285
7,KEYWORD,KEYWORD,1,9726f55f7b9c93ae2d6ce0d496d5033c39fe5302
8,KEYWORD,KEYWORD,1,11b3f41654cce7a4b754d325af064d3b4d86b533
9,KEYWORD,KEYWORD,1,872eab3ff582ceb93796aecce19250754cbe0d97


## 5) Workset prioritario y colapso por norma
Tomamos **solo** las combinaciones `PRIORITY_BUCKETS`, ordenamos y **quitamos duplicados** por `group_norma` (conservando el de mayor score).

In [14]:
work = df[df['bucket'].isin(PRIORITY_BUCKETS)].copy()
print('Workset (prioritario) tamaño:', len(work))

work = work.sort_values(['group_norma','score','rand'], ascending=[True, False, True])
work_nodup = work.drop_duplicates(subset=['group_norma'], keep='first').copy()
print('Tras colapsar por norma:', len(work_nodup))
work_nodup['bucket'].value_counts()

Workset (prioritario) tamaño: 6932
Tras colapsar por norma: 2321


,count
bucket,
VERBO;KEYWORD,2047
VERBO;KEYWORD;NORMAR,235
VERBO;NORMAR,39


## 6) Top-K por PDF (control local)
Dentro de `work_nodup`, mantenemos a lo sumo **TOPK_PER_PDF** por `origen_pdf`, priorizando por `score`.

In [15]:
work_nodup = work_nodup.sort_values(['origen_pdf','score','rand'], ascending=[True, False, True])
keep_topk = work_nodup.groupby('origen_pdf', as_index=False, group_keys=False).head(TOPK_PER_PDF)
print('Tras Top-K por PDF:', len(keep_topk))
keep_topk['bucket'].value_counts()

Tras Top-K por PDF: 2290


,count
bucket,
VERBO;KEYWORD,2016
VERBO;KEYWORD;NORMAR,235
VERBO;NORMAR,39


## 7) Cuotas por combinación para llegar a `TARGET_SIZE`
Asignamos cuotas por `bucket` y seleccionamos hasta completar el objetivo (si el work disponible no alcanza, tomamos todo lo disponible en ese bucket).

In [16]:
# === 7) Selección con prioridad a NORMAR ===
# Preseleccionar todo lo que tenga NORMAR
with_normar = keep_topk[keep_topk['bucket'].isin(['VERBO;KEYWORD;NORMAR','VERBO;NORMAR'])] \
                .sort_values(['score','rand'], ascending=[False, True])
selected = with_normar.copy()

# Resto disponible (sin NORMAR)
remaining = keep_topk.drop(with_normar.index).copy()

# Objetivo restante hasta TARGET_SIZE
remaining_target = max(0, TARGET_SIZE - len(selected))

# Del resto tomamos VERBO;KEYWORD por score (podés ajustar si agregás otros buckets)
sub = remaining[remaining['bucket'] == 'VERBO;KEYWORD'] \
        .sort_values(['score','rand'], ascending=[False, True])
pick = sub.head(remaining_target)
selected = pd.concat([selected, pick], axis=0, ignore_index=False)

# Si aún faltan filas y quedara algo más en 'remaining', completamos por score
if len(selected) < TARGET_SIZE and len(remaining) > 0:
    faltan = TARGET_SIZE - len(selected)
    tail = remaining.drop(pick.index, errors='ignore') \
                    .sort_values(['score','rand'], ascending=[False, True]) \
                    .head(faltan)
    selected = pd.concat([selected, tail], axis=0, ignore_index=False)

# Colapso extra por si se coló algún duplicado de norma
selected = selected.drop_duplicates(subset=['group_norma'], keep='first')

print('Seleccionados (final):', len(selected))
print(selected['bucket'].value_counts())

Seleccionados (final): 2000
bucket
VERBO;KEYWORD           1726
VERBO;KEYWORD;NORMAR     235
VERBO;NORMAR              39
Name: count, dtype: int64


## 8) Safety sample (opcional)
Tomamos una muestra aleatoria de buckets **no** prioritarios (p.ej. `KEYWORD;NORMAR` o `solo KEYWORD`) para no perder **casos raros**.
Si no querés safety sample, poné `INCLUDE_SAFETY_SAMPLE = False` en la celda de Configuración.

In [ ]:
if INCLUDE_SAFETY_SAMPLE and SAFETY_SIZE > 0:
    non_priority = df[~df['bucket'].isin(PRIORITY_BUCKETS)].copy()
    already = set(selected.index)
    non_priority = non_priority[~non_priority.index.isin(already)]
    np.random.seed(42)
    safety = non_priority.sample(n=min(SAFETY_SIZE, len(non_priority)), random_state=42)
    selected = pd.concat([selected, safety], axis=0, ignore_index=False)
    print('Se añadieron a safety:', len(safety))
else:
    print('Safety sample desactivado.')

selected = selected.drop_duplicates(subset=['group_norma'], keep='first')
print('Tras safety y colapso extra por norma:', len(selected))
selected['bucket'].value_counts()

## 9) Exportes
- `etiquetas_v1_TRIAGE.csv`: el lote **para etiquetar**.
- `etiquetas_v1_TRIAGE_pool.csv`: **todo lo demás** (no se pierde nada).

In [18]:
os.makedirs(os.path.dirname(CSV_TRIAGE), exist_ok=True)
selected_out = selected.copy()
pool_out = df[~df.index.isin(selected.index)].copy()

selected_out.to_csv(CSV_TRIAGE, index=False)
pool_out.to_csv(CSV_POOL, index=False)

print('Guardado TRIAGE →', CSV_TRIAGE)
print('Guardado POOL   →', CSV_POOL)
print('Tamaños → TRIAGE:', len(selected_out), '| POOL:', len(pool_out))

Guardado TRIAGE → /content/drive/MyDrive/IA/Proyectos/Análisis Boletín Oficial/boletin-ml/data/labels/etiquetas_v1_TRIAGE.csv
Guardado POOL   → /content/drive/MyDrive/IA/Proyectos/Análisis Boletín Oficial/boletin-ml/data/labels/etiquetas_v1_TRIAGE_pool.csv
Tamaños → TRIAGE: 2000 | POOL: 25250


## 10) Chequeos rápidos
- Conteos por bucket en TRIAGE.
- Duplicados por `group_norma`.
- Top por PDFs más representados.

In [19]:
tri = pd.read_csv(CSV_TRIAGE, dtype=str, keep_default_na=False)
print('TRIAGE size:', len(tri))
print('\nBuckets:')
print(tri['bucket'].value_counts())

dup_norma = tri['group_norma'].value_counts()
print('\nRepetidos por group_norma (>1):', (dup_norma > 1).sum())

print('\nTop 10 PDFs con más filas:')
print(tri['origen_pdf'].value_counts().head(10))

TRIAGE size: 2000

Buckets:
bucket
VERBO;KEYWORD           1726
VERBO;KEYWORD;NORMAR     235
VERBO;NORMAR              39
Name: count, dtype: int64

Repetidos por group_norma (>1): 0

Top 10 PDFs con más filas:
origen_pdf
20201228.pdf    20
20181022.pdf    20
20211102.pdf    20
20230810.pdf    20
20231213.pdf    19
20210331.pdf    19
20191015.pdf    19
20180710.pdf    18
20190301.pdf    18
20221202.pdf    18
Name: count, dtype: int64
